# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime

In [2]:
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

## Getting the Metadata from all the MED-PC Recording Files

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [5]:
all_med_pc_file = glob.glob("./data/clock_recording/*.txt")

- Use this instead if you're using you're own data

In [6]:
all_med_pc_file[:10]

['./data/clock_recording/2022-06-29_19h47m_Subject .txt',
 './data/clock_recording/2022-06-29_19h52m_Subject .txt']

- Example of what the MED-PC Recording file looks like

In [7]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-06-29_19h47m_Subject .txt







Start Date: 06/29/22

End Date: 06/29/22

Subject: 

Experiment: 

Group: 

Box: 1

Start Time: 19:47:27

End Time: 19:51:48

MSN: iwata_clock_test

A:    2799.000

D:    8000.000

F:    2000.000

L:       0.000

O:       0.000

T:     258.280

U:     230.010



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [8]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [9]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/clock_recording/2022-06-29_19h47m_Subject .txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-06-29_19h47m_Subject .txt', 'Start Date': '06/29/22', 'End Date': '06/29/22', 'Subject': '', 'Experiment': '', 'Group': '', 'Box': '1', 'Start Time': '19:47:27', 'End Time': '19:51:48', 'MSN': 'iwata_clock_test'}


## Making a Dataframe out of the Metadata

In [10]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [11]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/clock_recording/2022-06-29_19h47m_Subje...,C:\MED-PC\Data\2022-06-29_19h47m_Subject .txt,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test
1,./data/clock_recording/2022-06-29_19h52m_Subje...,C:\MED-PC\Data\2022-06-29_19h52m_Subject .txt,06/29/22,06/29/22,,,,1,19:52:36,19:56:56,iwata_clock_test


In [12]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/clock_recording/2022-06-29_19h47m_Subje...,C:\MED-PC\Data\2022-06-29_19h47m_Subject .txt,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test
1,./data/clock_recording/2022-06-29_19h52m_Subje...,C:\MED-PC\Data\2022-06-29_19h52m_Subject .txt,06/29/22,06/29/22,,,,1,19:52:36,19:56:56,iwata_clock_test


- Just getting the numbers out of the column that contains the cage information

In [13]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: x.strip("Cage").strip())

In [14]:
# How many files there are for each subject
metadata_df.groupby("Subject").count()

,index,File,Start Date,End Date,Experiment,Group,Box,Start Time,End Time,MSN,cage
Subject,,,,,,,,,,,
,2,2,2,2,2,2,2,2,2,2,2


## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [15]:
with open("./scripts/levelNP_CS_reward_laserepochON1st_noshock.MPC") as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\v3 stop tone with poke

\v3.2 monitor port entries AND exits



\INPUTS

^port = 8



\OUTPUTS

^fan = 16

^houselight = 11

^tone1 = 2

^tone2 = 3

^tone3 = 4

^tone4 = 5

^pump = 9

^whitenoise = 1

^laser=10

^csout = 5

^peout = 15

^cs1out = 12

^cs2out = 13

^cs3out = 14



\EXP SETTINGS

^ncsNoShock = 0

^initCS1trials = 3



\ARRAYS

DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity



LIST V = 30", 30", 30", 30", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 120", 80", 95", 80", 90", 80", 75", 100", 95", 90", 80", 90", 100", 90", 120", 90", 85", 90", 95", 90", 80", 90", 100", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90", 90"

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [16]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-06-29_19h47m_Subject .txt







Start Date: 06/29/22

End Date: 06/29/22

Subject: 

Experiment: 

Group: 

Box: 1

Start Time: 19:47:27

End Time: 19:51:48

MSN: iwata_clock_test

A:    2799.000

D:    8000.000

F:    2000.000

L:       0.000

O:       0.000

T:     258.280

U:     230.010



- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [17]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

In [18]:
concatted_medpc_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,46.71,64.0,399.0,0.0,60.01,30.0,48.0,19.0,47.12,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
1,47.29,144.0,399.0,0.0,140.01,49.0,49.0,19.0,47.60,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
2,47.70,234.0,399.0,0.0,230.01,20.0,51.0,19.0,47.94,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
3,65.21,NaN,NaN,0.0,8000.00,0.0,0.0,0.0,65.77,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
4,66.58,NaN,NaN,0.0,9000.00,0.0,0.0,0.0,67.25,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...


In [19]:
concatted_medpc_df.dropna(subset="(G)CS_secondspresentation")

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,46.71,64.0,399.0,0.0,60.01,30.0,48.0,19.0,47.12,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
1,47.29,144.0,399.0,0.0,140.01,49.0,49.0,19.0,47.60,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
2,47.70,234.0,399.0,0.0,230.01,20.0,51.0,19.0,47.94,1.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
3,65.21,NaN,NaN,0.0,8000.00,0.0,0.0,0.0,65.77,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
4,66.58,NaN,NaN,0.0,9000.00,0.0,0.0,0.0,67.25,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h47m_Subje...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2497,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2498,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...
2499,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,20220629,,./data/clock_recording/2022-06-29_19h52m_Subje...


- Getting the cage numbers and the dates so that we can include it in the file name

In [20]:
# removing blank spaces
cage_numbers = [number for number in metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [21]:
cage_numbers_for_title

''

In [22]:
# Getting the first and last recording date to get a range
earliest_date = concatted_medpc_df["date"].min()
latest_date = concatted_medpc_df["date"].max()

In [23]:
earliest_date

'20220629'

In [24]:
latest_date

'20220629'

- Making necessary directories

In [25]:
output_directory = "./proc/extracted_recording_data_and_metadata/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [26]:
output_directory

'./proc/extracted_recording_data_and_metadata/cage__date_20220629_20220629'

In [27]:
os.makedirs(output_directory, exist_ok=True)

In [28]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [29]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

- Combining the recording and the metadata into one dataframe

In [30]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index')

In [31]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,46.71,64.0,399.0,0.0,60.01,30.0,48.0,19.0,47.12,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
1,47.29,144.0,399.0,0.0,140.01,49.0,49.0,19.0,47.60,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
2,47.70,234.0,399.0,0.0,230.01,20.0,51.0,19.0,47.94,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
3,65.21,NaN,NaN,0.0,8000.00,0.0,0.0,0.0,65.77,0.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
4,66.58,NaN,NaN,0.0,9000.00,0.0,0.0,0.0,67.25,0.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,


In [32]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

# Making a column for the time

In [33]:
cs_time_df = recording_and_metadata_df[recording_and_metadata_df["(G)CS_secondspresentation"] != 0.0].dropna()

In [34]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,46.71,64.0,399.0,0.0,60.01,30.0,48.0,19.0,47.12,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
1,47.29,144.0,399.0,0.0,140.01,49.0,49.0,19.0,47.60,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
2,47.70,234.0,399.0,0.0,230.01,20.0,51.0,19.0,47.94,1.0,...,06/29/22,06/29/22,,,,1,19:47:27,19:51:48,iwata_clock_test,
2541,67.32,64.0,399.0,0.0,60.01,39.0,53.0,19.0,67.84,1.0,...,06/29/22,06/29/22,,,,1,19:52:36,19:56:56,iwata_clock_test,
2542,68.76,144.0,399.0,0.0,140.01,59.0,54.0,19.0,69.34,1.0,...,06/29/22,06/29/22,,,,1,19:52:36,19:56:56,iwata_clock_test,
2543,69.79,234.0,399.0,0.0,230.01,29.0,56.0,19.0,70.16,1.0,...,06/29/22,06/29/22,,,,1,19:52:36,19:56:56,iwata_clock_test,


In [35]:
cs_time_df.columns

Index(['(P)Portentry', '(Q)USdelivery', '(R)UStime', '(W)ITIvalues',
       '(S)CSpresentation', '(G)CS_secondspresentation',
       '(H)CS_minutespresentation', '(I)CS_hourspresentation', '(N)Portexit',
       '(K)CStype', '(B)shockintensity', 'date', 'subject', 'file_path',
       'index', 'File', 'Start Date', 'End Date', 'Subject', 'Experiment',
       'Group', 'Box', 'Start Time', 'End Time', 'MSN', 'cage'],
      dtype='object')

In [36]:
cs_time_df["cs_second"] = cs_time_df["(G)CS_secondspresentation"].astype(int).astype(str)
cs_time_df["cs_minute"] = cs_time_df["(H)CS_minutespresentation"].astype(int).astype(str)
cs_time_df["cs_hour"] = cs_time_df["(I)CS_hourspresentation"].astype(int).astype(str)


In [37]:
cs_time_df["cs_time_str"] = cs_time_df.apply(lambda x: ":".join([x["cs_hour"], x["cs_minute"],  x["cs_second"]]), axis=1)

In [38]:
cs_time_df["cs_time_object"] = cs_time_df["cs_time_str"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


In [39]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(G)CS_secondspresentation,(H)CS_minutespresentation,(I)CS_hourspresentation,(N)Portexit,(K)CStype,...,Box,Start Time,End Time,MSN,cage,cs_second,cs_minute,cs_hour,cs_time_str,cs_time_object
0,46.71,64.0,399.0,0.0,60.01,30.0,48.0,19.0,47.12,1.0,...,1,19:47:27,19:51:48,iwata_clock_test,,30,48,19,19:48:30,1900-01-01 19:48:30
1,47.29,144.0,399.0,0.0,140.01,49.0,49.0,19.0,47.60,1.0,...,1,19:47:27,19:51:48,iwata_clock_test,,49,49,19,19:49:49,1900-01-01 19:49:49
2,47.70,234.0,399.0,0.0,230.01,20.0,51.0,19.0,47.94,1.0,...,1,19:47:27,19:51:48,iwata_clock_test,,20,51,19,19:51:20,1900-01-01 19:51:20
2541,67.32,64.0,399.0,0.0,60.01,39.0,53.0,19.0,67.84,1.0,...,1,19:52:36,19:56:56,iwata_clock_test,,39,53,19,19:53:39,1900-01-01 19:53:39
2542,68.76,144.0,399.0,0.0,140.01,59.0,54.0,19.0,69.34,1.0,...,1,19:52:36,19:56:56,iwata_clock_test,,59,54,19,19:54:59,1900-01-01 19:54:59
2543,69.79,234.0,399.0,0.0,230.01,29.0,56.0,19.0,70.16,1.0,...,1,19:52:36,19:56:56,iwata_clock_test,,29,56,19,19:56:29,1900-01-01 19:56:29


# Importing Video Data

In [40]:
video_time_01 = pd.read_csv("./data/clock_recording/clock_test_time_2022-06-29T19_47_18.csv", header=None)
video_time_02 = pd.read_csv("./data/clock_recording/clock_test_time_2022-06-29T19_52_59.csv", header=None)

In [41]:
video_time_01["time"] = video_time_01[1].apply(lambda x: x.split()[-1].split(".")[0])
video_time_02["time"] = video_time_02[1].apply(lambda x: x.split()[-1].split(".")[0])

In [42]:
video_time_01

,0,1,time
0,(6/29/2022 11:47:20 PM,862); 2022-06-29 19:47:20.191000,19:47:20
1,(6/29/2022 11:47:20 PM,894); 2022-06-29 19:47:20.222000,19:47:20
2,(6/29/2022 11:47:20 PM,926); 2022-06-29 19:47:20.254000,19:47:20
3,(6/29/2022 11:47:20 PM,958); 2022-06-29 19:47:20.287000,19:47:20
4,(6/29/2022 11:47:21 PM,6); 2022-06-29 19:47:20.335000,19:47:20
...,...,...,...
8007,(6/29/2022 11:51:51 PM,439); 2022-06-29 19:51:50.734000,19:51:50
8008,(6/29/2022 11:51:51 PM,486); 2022-06-29 19:51:50.782000,19:51:50
8009,(6/29/2022 11:51:51 PM,518); 2022-06-29 19:51:50.813000,19:51:50
8010,(6/29/2022 11:51:51 PM,550); 2022-06-29 19:51:50.845000,19:51:50


In [43]:
video_01_start_time = video_time_01["time"][0]
video_02_start_time = video_time_02["time"][0]

In [44]:
video_01_start_time

'19:47:20'

In [45]:
video_02_start_time

'19:53:01'

In [46]:
datetime_object_01 = datetime.strptime(video_01_start_time, '%H:%M:%S')
datetime_object_02 = datetime.strptime(video_02_start_time, '%H:%M:%S')

In [47]:
datetime_object_01

datetime.datetime(1900, 1, 1, 19, 47, 20)

In [48]:
cs_time_df["cs_time_object"][0]

Timestamp('1900-01-01 19:48:30')

In [49]:
start_second_01 = cs_time_df["cs_time_object"][0] - datetime_object_01 

In [50]:
start_second_01.seconds

70

# Trimming the video

In [51]:
video_01 = VideoFileClip("./data/clock_recording/clock_test_recording_2022-06-29T19_47_20.avi")
video_02 = VideoFileClip("./data/clock_recording/clock_test_recording_2022-06-29T19_53_02.avi")

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [55]:
video_01.subclip(start_second_01.seconds)